In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/university-students-complaints-and-reports/Datasetprojpowerbi.csv


In [2]:
import pandas as pd
import string
import nltk
from nltk.corpus import stopwords
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
nltk.download("punkt")
from nltk.stem import PorterStemmer
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np

df = pd.read_csv("/kaggle/input/university-students-complaints-and-reports/Datasetprojpowerbi.csv")

df["Reports_lower"] = df["Reports"].str.lower()

translator = str.maketrans(" "," ",string.punctuation)
df["Reports_nopunct"] = df["Reports_lower"].str.translate(translator)

english_stopwords = set(stopwords.words("english"))
custom_stopwords = {"im","really","student","like","feel","ive"}
combined_stopwords = english_stopwords.union(custom_stopwords)
def remove_stopwords(text):
    words = word_tokenize(text.lower())
    improved_text = [word for word in words if word not in combined_stopwords and word not in translator and not word.isdigit()]
    return " ".join(improved_text)

df["cleaned_Reports"] = df["Reports_nopunct"].apply(remove_stopwords)

stemmer = PorterStemmer()
def stem_text(text):
    words = word_tokenize(text)
    stemmed_words = [stemmer.stem(word) for word in words]
    return " ".join(stemmed_words)

df["stemmed_Reports"] = df["cleaned_Reports"].apply(stem_text)

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["stemmed_Reports"])
tfidf_df = pd.DataFrame(X.toarray(), columns = vectorizer.get_feature_names_out())

n_topics = 10
lda = LatentDirichletAllocation(n_components=n_topics, random_state=42)
lda.fit(X)
feature_names = vectorizer.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[-5:]]))

doc_topic_dist = lda.transform(X)
topic_prevalence = doc_topic_dist.sum(axis=0)
topic_df = pd.DataFrame({"Topic": range(lda.n_components), "Prevalence": topic_prevalence})

topic_df = topic_df.sort_values(by="Prevalence", ascending=False)

print(doc_topic_dist[0])
print(topic_df)


[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
Topic #0:
money applic caus technic difficulti
Topic #1:
make campu medic afford option
Topic #2:
offer dine opportun food job
Topic #3:
need ix titl injuri univers
Topic #4:
struggl academ work time balanc
Topic #5:
unfair elig unabl scholarship athlet
Topic #6:
affect health anxieti mental stress
Topic #7:
cours research materi limit access
Topic #8:
resourc support univers cultur student
Topic #9:
opportun career internship academ job
[0.02102602 0.02102609 0.02102365 0.02102396 0.02102427 0.0210235
 0.02102737 0.81077461 0.02102482 0.02102572]
   Topic  Prevalence
8      8  183.292846
1      1  171.121513
7      7  111.938883
2      2  103.044120
6      6   99.680179
4      4   82.681628
9      9   78.001224
3      3   66.796626
0      0   57.576